# Numerical norm bounds for cartpole 

For a cartpole system with state $y = \begin{bmatrix}x & \dot{x} & \varphi & \dot{\varphi} \end{bmatrix}^T$ we have 
|
\begin{equation}
\dot{y} = \begin{bmatrix} 
\dot{x} \\
\frac{u + m_p \sin\varphi (l \dot{\varphi}^2 + g \cos\varphi)}{m_c + m_p \sin^2\varphi} \\
\dot{\varphi} \\
\frac{-u\cos\varphi - m_p l \dot{\varphi}^2 \cos\varphi \sin\varphi - (m_c+m_p)g\sin\varphi}{l(m_c + m_p \sin^2\varphi)}
\end{bmatrix}.
\end{equation}

Evaluating the corresponding Jacobian at 0 (in the state and action space) yields:
\begin{equation}
\nabla_{\begin{bmatrix} y & u \end{bmatrix}^T} f(0)\begin{bmatrix} y \\ u \end{bmatrix} = \begin{bmatrix} \dot{x} \\ \frac{-m_p g \varphi + u}{m_c}\\
\dot{\varphi} \\
\frac{g(m_c + m_p)\varphi - u}{l m_c}
\end{bmatrix}
\end{equation}

We want to find an NLDI of the form
\begin{equation}
\dot{y} = \nabla_{\begin{bmatrix} y & u \end{bmatrix}^T} f(0) \begin{bmatrix} y \\ u \end{bmatrix} + I p, \;\; \|p\| \leq \|Cy + Du\|
\end{equation}

To find $C$ and $D$," we determine an entry-wise norm bound. That is, for $i=1,\ldots,4$, we want to find $F_i$ such that for all $y$ such that $y_{\text{min}} \leq y \leq y_{\text{max}}$, and all $u$ such that $u_{\text{min}} \leq u \leq u_{\text{max}}$:
\begin{equation}
(\nabla f_i(0) \begin{bmatrix} y \\ u \end{bmatrix} - \dot{y}_i)^2 \leq \begin{bmatrix} y \\ u \end{bmatrix}^T F_i \begin{bmatrix} y \\ u \end{bmatrix}
\end{equation}
and then write
\begin{equation}
\|\dot{y} - \nabla f(0)\begin{bmatrix} y \\ u \end{bmatrix}\|_2 \leq \|\begin{bmatrix} F_1^{1/2} \\ F_2^{1/2} \\ F_3^{1/2} \\ F_4^{1/2} \end{bmatrix} \begin{bmatrix} y \\ u \end{bmatrix} \| = \|\begin{bmatrix} F_1^{1/2} \\ F_2^{1/2} \\ F_3^{1/2} \\ F_4^{1/2} \end{bmatrix}_{:, :n} y + \begin{bmatrix} F_1^{1/2} \\ F_2^{1/2} \\ F_3^{1/2} \\ F_4^{1/2} \end{bmatrix}_{:, n:n+m} u \|,
\end{equation}
where $n = \dim(y) = 4$, $m = \dim(u) = 1$.


In [112]:
import numpy as np
import cvxpy as cp
import scipy.linalg as sla

In [113]:
g = 9.81
l = 1
mc = 1
mp = 1

## Define max and min values 

In [114]:
# State is: [y, u] = [x, xdot, phi, phidot, u]^T
s_max = np.array([1.2, 1.0, 0.1, 1.0, 10])
s_min = np.array([-1.2, -1.0, -0.1, -1.0, -10])

x_max, xdot_max, phi_max, phidot_max, u_max = s_max
x_min, xdot_min, phi_min, phidot_min, u_min = s_min

n = 4
m = 1
nm = n+m
x_idx, xdot_idx, phi_idx, phidot_idx, u_idx = range(nm)

## Find element-wise bounds 

### $f_1$

No error -- linearization is the same as original

### $f_2$ 

In [115]:
gridnum = 50
phi = np.linspace(phi_min, phi_max, gridnum)
phidot = np.linspace(phidot_min, phidot_max, gridnum)
u = np.linspace(u_min, u_max, gridnum)

Phi, Phidot, U = np.meshgrid(phi, phidot, u)

v2 = np.ravel(( (-mp*g*Phi + U)/mc - 
                (U + mp*np.sin(Phi)*(l*Phidot**2 - g*np.cos(Phi)))/(mc + mp*np.sin(Phi)**2) )**2)
U2 = np.array([np.ravel(Phi*Phi), 
              np.ravel(Phidot*Phidot), 
              np.ravel(U*U),
              2*np.ravel(Phi*Phidot), 
              2*np.ravel(Phi*U), 
              2*np.ravel(Phidot*U)]).T

In [116]:
c2 = cp.Variable(6)
cp.Problem(cp.Minimize(cp.max(U2@c2 - v2)), [U2@c2 >= v2, c2[:3]>=0]).solve(verbose=True, solver=cp.MOSEK)



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 250003          
  Cones                  : 0               
  Scalar variables       : 7               
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 250003          
  Cones                  : 0               
  Scalar variables       : 7               
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the dual        
Optimizer  - Constraints            : 7
Optimizer  - Cones                  : 0
Optimizer  - Scalar variables      

0.020355041002912407

In [117]:
c2 = c2.value
c2

array([ 1.73328132e+00,  1.28470585e-02,  2.62093606e-05,  5.11706867e-13,
       -6.42813415e-03,  1.55358430e-15])

In [118]:
C2 = np.zeros((nm,nm))
C2[phi_idx, phi_idx] = c2[0]/2
C2[phidot_idx, phidot_idx] = c2[1]/2
C2[u_idx, u_idx] = c2[2]/2
C2[phi_idx, phidot_idx] = c2[3]
C2[phi_idx, u_idx] = c2[4]
C2[phidot_idx, u_idx] = c2[5]
C2 += C2.T

In [119]:
np.linalg.eig(C2)[0]

array([1.28470585e-02, 1.73330516e+00, 2.36962697e-06, 0.00000000e+00,
       0.00000000e+00])

In [120]:
gam2 = np.real(sla.sqrtm(C2))
gam2

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.31653239e+00,
         3.57965622e-13, -4.87685586e-03],
       [ 0.00000000e+00,  0.00000000e+00,  3.57965622e-13,
         1.13344865e-01,  2.87142270e-14],
       [ 0.00000000e+00,  0.00000000e+00, -4.87685586e-03,
         2.87142270e-14,  1.55744585e-03]])

### $f_3$ 

No error -- linearization is the same as original

### $f_4$

In [121]:
gridnum = 50
phi = np.linspace(phi_min, phi_max, gridnum)
phidot = np.linspace(phidot_min, phidot_max, gridnum)
u = np.linspace(u_min, u_max, gridnum)
""
Phi, Phidot, U = np.meshgrid(phi, phidot, u)
""
v4 = np.ravel(( (g*(mc+mp)*Phi - U)/(l*mc) - 
                (U*np.cos(Phi) + mp*l*(Phidot**2)*np.cos(Phi)*np.sin(Phi) - (mc+mp)*g*np.sin(Phi))/(
                    -l*(mc + mp*np.sin(Phi)**2)) 
              )**2)
U4 = np.array([np.ravel(Phi*Phi), 
              np.ravel(Phidot*Phidot), 
              np.ravel(U*U),
              2*np.ravel(Phi*Phidot), 
              2*np.ravel(Phi*U), 
              2*np.ravel(Phidot*U)]).T

In [122]:
c4 = cp.Variable(6)
cp.Problem(cp.Minimize(cp.max(U4@c4 - v4)), [U4@c4 >= v4, c4[:3]>=0]).solve(verbose=True, solver=cp.MOSEK)



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 250003          
  Cones                  : 0               
  Scalar variables       : 7               
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 250003          
  Cones                  : 0               
  Scalar variables       : 7               
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the dual        
Optimizer  - Constraints            : 7
Optimizer  - Cones                  : 0
Optimizer  - Scalar variables      

0.032262717853947874

In [123]:
c4 = c4.value
c4

array([ 2.86341906e+00,  1.43252388e-02,  8.28430784e-05, -1.71779544e-12,
       -1.05858385e-02, -1.88719950e-14])

In [124]:
C4 = np.zeros((nm,nm))
C4[phi_idx, phi_idx] = c4[0]/2
C4[phidot_idx, phidot_idx] = c4[1]/2
C4[u_idx, u_idx] = c4[2]/2
C4[phi_idx, phidot_idx] = c4[3]
C4[phi_idx, u_idx] = c4[4]
C4[phidot_idx, u_idx] = c4[5]
C4 += C4.T

In [125]:
np.linalg.eig(C4)[0]

array([1.43252388e-02, 2.86345820e+00, 4.37074558e-05, 0.00000000e+00,
       0.00000000e+00])

In [126]:
gam4 = np.real(sla.sqrtm(C4))
gam4

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.69215254e+00,
        -9.48768893e-13, -6.23141107e-03],
       [ 0.00000000e+00,  0.00000000e+00, -9.48768893e-13,
         1.19688090e-01, -1.96197871e-13],
       [ 0.00000000e+00,  0.00000000e+00, -6.23141107e-03,
        -1.96197871e-13,  6.63419885e-03]])

## Final system 

In [127]:
A = np.array([[0, 1, 0, 0],
             [0, 0, -mp*g/mc, 0],
             [0, 0, 0, 1],
             [0, 0, g*(mc+mp)/(l*mc), 0]])
B = np.expand_dims(np.array([0, 1/mc, 0, -1/(l*mc)]), 1)

# G_inv = np.array([[1, 0, 0, 0],
#                   [0, a2[0], 0, a2[1]],
#                   [0, 0, 1, 0],
#                   [0, a4[0], 0, a4[1]]])
# G = np.linalg.inv(G_inv)
G = np.eye(4)

F = np.vstack([gam2, gam4])
C = F[:,:n]
D = F[:,n:]

### Check correctness 

In [128]:
from IPython.core.debugger import set_trace

In [129]:
def xdot_f(state, u):
    # x = state[:, 0]
    x_dot = state[:, 1]
    theta = state[:, 2]
    theta_dot = state[:, 3]
    u = u.flatten()

    sin_theta = np.sin(theta)
    cos_theta = np.cos(theta)
    temp = 1/(mc + mp * (sin_theta * sin_theta))
    x_ddot = temp * (u + mp * np.sin(theta) * (l * (theta_dot**2)
                                                        - g * cos_theta))
    theta_ddot = -(1/l) * temp * (u * cos_theta
                                      + mp * l * (theta_dot**2) * cos_theta * sin_theta
                                      - (mc + mp) * g * sin_theta)
    return np.vstack([x_dot, x_ddot, theta_dot, theta_ddot]).T

In [130]:
prop = np.random.random((10000, nm))
rand_ss = s_max*prop + s_min*(1-prop)
rand_ys = rand_ss[:,:n]
rand_us = rand_ss[:,n:]

fx = xdot_f(rand_ys, rand_us)
# print(np.linalg.norm((fx - rand_ys@A.T - rand_us@B.T)@np.linalg.inv(G).T, axis=1) <= \
#       np.linalg.norm(rand_ys@C.T + rand_us@D.T, axis=1))
print((np.linalg.norm((fx - rand_ys@A.T - rand_us@B.T)@np.linalg.inv(G).T, axis=1) <= \
      np.linalg.norm(rand_ys@C.T + rand_us@D.T, axis=1)).all())

ratio = np.linalg.norm(rand_ys@C.T + rand_us@D.T, axis=1)/np.linalg.norm(
    (fx - rand_ys@A.T - rand_us@B.T)@np.linalg.inv(G).T, axis=1)
print(ratio.max())
print(ratio.mean())
print(np.median(ratio))

True
84078.10099634247
69.84146506508074
5.992302081027291


### Save 

In [131]:
np.save('A.npy', A)
np.save('B.npy', B)
np.save('G.npy', G)
np.save('C.npy', C)
np.save('D.npy', D)

## Check if robust LQR solves 

In [132]:
import scipy.linalg as la

In [133]:
Q = np.random.randn(n, n)
Q = Q.T @ Q
# Q = np.eye(n)

R = np.random.randn(m, m)
R = R.T @ R
# R = np.eye(m)

In [134]:
alpha = 0.001

n, m = B.shape
wq = C.shape[0]

S = cp.Variable((n, n), symmetric=True)
Y = cp.Variable((m, n))
mu = cp.Variable()

R_sqrt = la.sqrtm(R)
f = cp.trace(S @ Q) + cp.matrix_frac(Y.T @ R_sqrt, S)

cons_mat = cp.bmat((
    (A @ S + S @ A.T + cp.multiply(mu, G @ G.T) + B @ Y + Y.T @ B.T + alpha * S, S @ C.T + Y.T @ D.T),
    (C @ S + D @ Y, -cp.multiply(mu, np.eye(wq)))
))
cons = [S >> 0, mu >= 1e-2] + [cons_mat << 0]

cp.Problem(cp.Minimize(f), cons).solve(solver=cp.MOSEK, verbose=True)
K = np.linalg.solve(S.value, Y.value.T).T



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 259             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 3               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 259             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 3               
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 237
Optimizer  - Cones                  : 1
Optimizer  - Scalar variables

In [135]:
print((rand_ys@K.T).max())
print((rand_ys@K.T).min())
print((rand_ys@K.T).mean())
print(np.median(rand_ys@K.T))
print(abs(rand_ys@K.T).mean())
print(np.median(abs(rand_ys@K.T)))

33.38321398489339
-33.42043056485856
0.014336330683547194
-0.047685520942526205
10.38160108794252
9.56790890554478


## Get a sense for u size in non-robust case 

In [136]:
import control

In [137]:
K, S, _ = control.lqr(A, B, Q, R)
K = np.array(-K)

In [138]:
print((rand_ys@K.T).max())
print((rand_ys@K.T).min())
print((rand_ys@K.T).mean())
print(np.median(rand_ys@K.T))
print(abs(rand_ys@K.T).mean())
print(np.median(abs(rand_ys@K.T)))

30.167968588820685
-30.21452996513471
0.015277241681704091
-0.019151745702437273
9.448202543340196
8.76269849972503


In [139]:
c2 = cp.Variable(6)
c4 = cp.Variable(6)
a2 = cp.Variable(2)
a4 = cp.Variable(2)
prob2 = cp.Problem(cp.Minimize(cp.max(U2@c2 - v2 * a2[0])), [U2@c2 >= v2 * a2[0] + v4 * a2[1], c2[:3]>=0, a2 >= 0, cp.sum(a2) == 1])
prob2.solve(verbose=True, solver=cp.MOSEK)
prob4 = cp.Problem(cp.Minimize(cp.max(U2@c4 - v4 * a4[1])), [U4@c4 >= v2 * a4[0] + v4 * a4[1], c4[:3]>=0, a4 >= 0, cp.sum(a4) == 1])
prob4.solve(verbose=True, solver=cp.MOSEK)
# cp.Problem(cp.Minimize(cp.max(U4@c4 - v4)), [U4@c4 >= v4, c4[:3]>=0]).solve(verbose=True, solver=cp.MOSEK)

a2 = a2.value
a4 = a4.value



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 250006          
  Cones                  : 0               
  Scalar variables       : 9               
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 250006          
  Cones                  : 0               
  Scalar variables       : 9               
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the dual        
Optimizer  - Constraints            : 9
Optimizer  - Cones                  : 0
Optimizer  - Scalar variables      

In [140]:
C_old = np.load('C.npy')
C - C_old

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])